In [1]:
import numpy as np
import scipy
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.utils import np_utils
#from sklearn.cross_validation import StratifiedKFold

Using TensorFlow backend.


In [2]:
PATH = os.getcwd()
print (PATH)

/home/danquxunhuan/course/prml/lab2/Indian_pines_classification


In [3]:
# Global Variables
windowSize = 5
numPCAcomponents = 30
testRatio = 0.25

# Load Training Dataset

In [4]:
X_train = np.load("./predata/XtrainWindowSize" 
                  + str(windowSize) + "PCA" + str(numPCAcomponents) + "testRatio" + str(testRatio)  + ".npy")

y_train = np.load("./predata/ytrainWindowSize" 
                  + str(windowSize) + "PCA" + str(numPCAcomponents) + "testRatio" + str(testRatio) + ".npy")

In [5]:
# Reshape into (numberofsumples, channels, height, width)
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[3], X_train.shape[1], X_train.shape[2]))

In [6]:
# convert class labels to on-hot encoding
y_train = np_utils.to_categorical(y_train)

In [7]:
# Define the input shape 
input_shape= X_train[0].shape
print(input_shape)

(30, 5, 5)


In [8]:
# number of filters
C1 = 3*numPCAcomponents

In [9]:
# Define the model
model = Sequential()

model.add(Conv2D(C1, (3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(3*C1, (3, 3), activation='relu'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(6*numPCAcomponents, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='softmax'))

In [10]:
sgd = SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [11]:
model.fit(X_train, y_train, batch_size=32, epochs=30)

Epoch 1/30
30164/30164 [==============================] - 6s 201us/step - loss: 2.4987 - acc: 0.2203
Epoch 2/30
30164/30164 [==============================] - 5s 166us/step - loss: 1.8668 - acc: 0.4570
Epoch 3/30
30164/30164 [==============================] - 5s 167us/step - loss: 1.3288 - acc: 0.6234
Epoch 4/30
30164/30164 [==============================] - 5s 166us/step - loss: 0.9547 - acc: 0.7213
Epoch 5/30
30164/30164 [==============================] - 5s 168us/step - loss: 0.7392 - acc: 0.7714
Epoch 6/30
30164/30164 [==============================] - 5s 166us/step - loss: 0.6089 - acc: 0.8083
Epoch 7/30
30164/30164 [==============================] - 5s 165us/step - loss: 0.5222 - acc: 0.8346
Epoch 8/30
30164/30164 [==============================] - 5s 165us/step - loss: 0.4563 - acc: 0.8541
Epoch 9/30
30164/30164 [==============================] - 5s 165us/step - loss: 0.4083 - acc: 0.8690
Epoch 10/30
30164/30164 [==============================] - 5s 164us/step - loss: 0.3696 - a

In [12]:
import h5py
from keras.models import load_model

In [13]:
model.save('./model/HSI_model_epochs30.h5')